In [138]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [139]:
# CSV einlesen mit robustem Parser
df = pd.read_csv("Ladesaeulenregister_BNetzA_2025-05-07.CSV", sep=';', encoding='ISO-8859-1', dtype=str)

In [140]:
# Nur relevante Spalten extrahieren
df = df[["Breitengrad", "Längengrad", "Ort", "Straße", "Hausnummer"]].copy()

In [141]:
# Fehlende Werte entfernen
df = df.dropna(subset=["Breitengrad", "Längengrad"])

In [142]:
# Funktion: Nur Werte behalten, die nur aus Ziffern und maximal einem Punkt bestehen
def koordinaten_bereinigen(df):
    def ist_gueltig(val):
        val = str(val).replace('.', '')
        return val.isdigit()

    df = df[df["Breitengrad"].apply(ist_gueltig)]
    df = df[df["Längengrad"].apply(ist_gueltig)]
    return df

df = koordinaten_bereinigen(df)

In [143]:
# Formatierungsfunktionen
def format_breitengrad(val):
    val = str(val).replace('.', '')
    val = val.ljust(8, '0')[:8]
    return f"{val[:-6]}.{val[-6:]}"  # z.B. 51805270 → 51.805270

def format_laengengrad(val):
    val = str(val).replace('.', '')
    if val.startswith("1"):
        val = val.ljust(8, '0')[:8]
    else:
        val = val.ljust(7, '0')[:7]
    return f"{val[:-6]}.{val[-6:]}"  # z.B. 10332870 → 10.332870

df["Breitengrad"] = df["Breitengrad"].apply(format_breitengrad)
df["Längengrad"] = df["Längengrad"].apply(format_laengengrad)

In [144]:
# Konvertieren in float
df["Breitengrad"] = df["Breitengrad"].astype(float)
df["Längengrad"] = df["Längengrad"].astype(float)

In [145]:
# Nur Koordinaten in Deutschland behalten
df = df[
    df["Breitengrad"].between(47.0, 55.1) &
    df["Längengrad"].between(5.5, 15.5)
]

print(f"Aktuell verbleibende Datensätze: {len(df)}")

Aktuell verbleibende Datensätze: 88575


In [146]:
# Karte erzeugen
m = folium.Map(location=[51.1657, 10.4515], zoom_start=6)
marker_cluster = MarkerCluster().add_to(m)

In [147]:
# Marker hinzufügen (hier: max. 10.000 für Performance)
for _, row in df.head(89000).iterrows():
    popup_text = f"{row.get('Ort', '')}, {row.get('Straße', '')} {row.get('Hausnummer', '')}"
    folium.Marker(
        location=[row["Breitengrad"], row["Längengrad"]],
        popup=popup_text
    ).add_to(marker_cluster)

In [148]:
# Karte speichern
m.save("ladesaeulen_karte.html")